In [ ]:
import pandas as pd
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from datetime import date, datetime, timedelta

now = datetime.now()
year_month_id = now.strftime("%Y%m")
s3_bucket_path = "s3://dataforgood-fb-data/demographic_csvs/"
folders = dbutils.fs.ls(s3_bucket_path)
for folder in folders:
    # Get only vietnam area
    s3_path = folder.path + '*lat_[0,20]*_lon_[90,120]*.csv'
    # s3_path = folder.path + '*.csv'
    # df = spark.read.format("csv").option("header", "true").option("recursiveFileLookup", "true").load(s3_path)
    # df = df.unpivot(['latitude', 'longitude'], df.columns[2], "population_type", "population")
    # df = df.withColumn("year_month_id", lit(year_month_id))
    # (df.write
    # .mode("overwrite")
    # .partitionBy(["year_month_id", "population_type"])
    # .option("overwriteSchema", "true")
    # .option("partitionOverwriteMode", "dynamic")
    # .save("/mnt/data/raw_area/Population_Density_Maps_MetaDFG/")
    # )


## Resolution for H3 Grids
> 

| H3 Resolution | Average Hexagon Area (km2) | Average Hexagon Edge Length (km) | Number of unique indexes |
| ------------- | -------------------------- | -------------------------------- | ------------------------ |
|0 |4,250,546.8477000|1,107.712591000|122|
|1 |607,220.9782429  |418.676005500|842|
|2 |86,745.8540347   |158.244655800|5,882|
|3 |12,392.2648621   |59.810857940|41,162|
|4 |1,770.3235517    |22.606379400|288,122|
|5 |252.9033645      |8.544408276|2,016,842|
|6 |36.1290521       |3.229482772|14,117,882|
|7 |5.1612932        |1.220629759|98,825,162|
|8 |0.7373276        |0.461354684|691,776,122|
|9 |0.1053325        |0.174375668|4,842,432,842|
|10|0.0150475        |0.065907807|33,897,029,882|
|11|0.0021496        |0.024910561|237,279,209,162|
|12|0.0003071        |0.009415526|1,660,954,464,122|
|13|0.0000439        |0.003559893|11,626,681,248,842|
|14|0.0000063        |0.001348575|81,386,768,741,882|
|15|0.0000009        |0.000509713|569,707,381,193,162|

From https://www.programmersought.com/article/30995709682/

In [ ]:
import geopandas
import json
from urllib.request import urlopen

url = 'https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/country_shapes/exports/geojson'
jsonData = urlopen(url).read().decode('utf-8')
data = geopandas.read_file(jsonData, where="cntry_name='Vietnam'")
data = data.explode(index_parts=True)
gjson = data.to_json()

In [ ]:
jsonObject = json.loads(gjson)
for i in jsonObject['features']:
    print(i['geometry'])
# print the keys and values
# for key in jsonObject:
#     value = jsonObject[key]
#     print("The key and value are ({}) = ({})".format(key, value))

{'type': 'Polygon', 'coordinates': [[[106.45821380615197, 11.665863037109398], [106.44830322265597, 11.709720611572196], [106.43773651122999, 11.793609619140597], [106.44857788085899, 11.817220687866197], [106.46135711669898, 11.884998321533299], [106.42117309570297, 11.965351104736296], [106.41954803466797, 11.972636222839396], [106.44538116455097, 11.982080459594696], [106.62868499755899, 11.971526145935098], [106.69857788085898, 11.962776184081996], [106.72608184814499, 11.973815917968796], [106.74710083007797, 12.0], [106.78919982910197, 12.073461532592898], [106.85693359374997, 12.067220687866197], [106.90734863281297, 12.064651489257797], [106.95583343505898, 12.074720382690396], [106.97470855712899, 12.082497596740797], [107.05275726318398, 12.139442443847699], [107.08193969726597, 12.177776336669996], [107.11080932617197, 12.215831756591898], [107.16276550292997, 12.273748397827298], [107.18997192382797, 12.287220001220797], [107.23858642578097, 12.303609848022498], [107.296371

In [ ]:
import h3
import pandas as pd
import pyspark.sql.functions as F
import reverse_geocoder as rg

from keplergl import KeplerGl
from pyspark.sql.functions import col

vietnam_geojson = [[[102.1175043471,8.2836024352],[109.6541254409,8.2836024352],[109.6541254409,23.4494630389],[102.1175043471,23.4494630389],[102.1175043471,8.2836024352]]]
def polyfill_geo_json(geojson, res):
    out = h3.polyfill(geojson, res, True)
    
    return out

res = 6
df = pd.DataFrame()
for i in jsonObject['features']:
    print(i['id'])
    r = pd.DataFrame(polyfill_geo_json(i['geometry'], res), columns=['h3_hex_id'])
    df = pd.concat([df, r])
df['h3_resolution'] = res
df['h3_hex_centroid'] = df.apply(lambda x: h3.h3_to_geo(x['h3_hex_id']), axis = 1)
df['geocode_reverse'] = rg.search(df['h3_hex_centroid'].values.tolist())
df['country'] = df['geocode_reverse'].apply(lambda x: x['cc'])
df['province'] = df['geocode_reverse'].apply(lambda x: x['admin1'])
df = spark.createDataFrame(df)
(df.write
.mode("overwrite")
.option("overwriteSchema", "true")
.option("overwriteSchema", "true")
.save("/mnt/data/raw_area/country_h3_index/")
)

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)


In [ ]:
%sql
CREATE TABLE IF NOT EXISTS STG.FACT_RAW_DEMOGRAPHIC_POPULATION USING DELTA LOCATION '/mnt/data/raw_area/Population_Density_Maps_MetaDFG/';
CREATE TABLE IF NOT EXISTS STG.COUNTRY_H3_INDEX USING DELTA LOCATION '/mnt/data/raw_area/country_h3_index/';